# Implementacja automatycznego różniczkowania w języku Julia

- Porównanie różniczkowania w przód i w tył funkcji jednej zmiennej
- Porównanie różniczkowania w przód i w tył funkcji wielu zmiennych
- Implementacja prostej sieci neuronowej


###### M.Foltys, Algorytmy w Inżynierii Danych. Czerwiec 2021

In [ ]:
# Sprawdzenie poprawności działania kernela Julii oraz dodanie bibliotek zewnętrznych
# Testowane na wersji Julii 1.5.3

using Plots # Tworzenie wykresów
using BenchmarkTools # Pomiar czasu wykonania kodu i alokowana pamięć
using DelimitedFiles # zapis do CSV 
import LinearAlgebra: diagm

versioninfo()

# Przygotowanie do różniczkowania w przód

#### Inicjalizacja liczb dualnych

In [ ]:
# zdefiniowanie struktury liczby Dualnej
# Taka liczba przechowuje wartość or wartość pochodnej
struct Dual{T <:Number} <:Number
  v::T
  dv::T
end

In [ ]:
# Część odpowiedzialna za przeciążenia podstawowych operatorów
import Base: +, -, *, /, ^
-(x::Dual) = Dual(-x.v, -x.dv)
+(x::Dual, y::Dual) = Dual( x.v + y.v, x.dv + y.dv)
-(x::Dual, y::Dual) = Dual( x.v - y.v, x.dv - y.dv)
*(x::Dual, y::Dual) = Dual( x.v * y.v, x.dv * y.v + x.v * y.dv)
/(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)
^(x::Dual, y::Dual) = Dual( x.v ^ y.v, x^y*log(x))

# Definicja podstawowych funkcji, gdy na wejściu dostaną liczbę dualną
import Base: abs, sin, cos, tan, exp, sqrt, isless
abs(x::Dual) = Dual(abs(x.v),sign(x.v)*x.dv)
sin(x::Dual) = Dual(sin(x.v), cos(x.v)*x.dv)
cos(x::Dual) = Dual(cos(x.v),-sin(x.v)*x.dv)
tan(x::Dual) = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
exp(x::Dual) = Dual(exp(x.v), exp(x.v)*x.dv)
log(x::Dual) = Dual(log(x.v), (1/x.v)*x.dv)
sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
softmax(x::Dual) = Dual(softmax(x.v), softmax_d(x.v)*x.dv)
isless(x::Dual, y::Dual) = x.v < y.v;

In [ ]:
# Funkcje pomocnicze ułatwiające pracę z liczbami dualnymi
import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

In [ ]:
# Promocja typów i konwersja
import Base: convert, promote_rule
convert(::Type{Dual{T}}, x::Dual) where T = Dual(convert(T, x.v), convert(T, x.dv))

convert(::Type{Dual{T}}, x::Number) where T = Dual(convert(T, x), zero(T))

convert(::Type{Dual{T}}, x::Float64) where T = Dual(convert(T, x), zero(T))

promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} = Dual{promote_type(T,R)}

In [ ]:
# Zdefiniowanie zarodka 
ϵ = Dual(0., 1.)

# Definiowanie funkcji jednej zmiennej
###### Funkcje te będą różniczkowane w kolejnych segmentach programu

In [ ]:
σ(x) = one(x) / (one(x) + exp(-x));
softmax(x) = exp.(x) ./ sum(exp.(x));
# definicja funkcji softmax to funkcja exponentalna podzielona przez sumę poszczególnych exponentali z x
tanh(x) = 2.0 / (one(x) +exp(-2.0x)) - one(x);
f(x) = x > zero(x) ? x : zero(x);
sink(x) = (sin(x)*sin(x)); #sinus kwadrat
sinexp(x) = sin(exp(x)); # sin(exp(x)^2))
RELU(x) = max(zero(x),x)

In [ ]:
# pochodna funkcji softmax nie jest prosta, do jej wyliczania konieczne jest utworzenie pipeline'a.
diagonal(m) = diagm(0 => vec(m))
# vec() przekształca macierz w jedną kolumnę, następnie za pomocą diagm tworzona jest macierz diagonalna.
# W resztę miejsc wpisywane są zera
softmax_d(x) = (softmax(x) |> diagonal) .- softmax(x) * (softmax(x) |> transpose);
# Wejście zostaje zamienione na macierz diagonalną, 
# następnie odejmowane są od niej wartości macierzy pomnożonej przez jej transpozycję

# Inicjalizacja funkcji do różniczkowania w tył
##### Zastosowano rozwiązanie implementujące sieć węzłów

In [ ]:
# niezbędniki do konstrukcji grafu
abstract type Node end
abstract type Operator end
abstract type LeafNode <: Node end

In [ ]:
# zmienna przechowująca wartość i pochodną
mutable struct Variable{T} <: LeafNode
    value::T
    grad::T
    Variable(val::T) where T          = new{T}(val, zero(val))
    Variable(val::T, grad::T) where T = new{T}(val, grad)
end

In [ ]:
struct Method{OT} <: Operator
    f::OT
end

struct Broadcasted{OT} <: Operator
    f::OT
end

In [ ]:
# reprezentacja operacji
struct ComputableNode{OT <: Operator, AT <: Tuple, KT <: NamedTuple} <: Node
    op::OT
    args::AT
    kwargs::KT
end
ComputableNode(op::Function, args, kwargs) = ComputableNode(Method(op), args, kwargs)
ComputableNode(op, args)                   = ComputableNode(op, args, NamedTuple())

In [ ]:
# CachedNode owija ComputableNode, zapamiętuje wynik i pozwala na jednorazowe wykonanie

mutable struct CachedNode{NT <: Node, OUT} <: Node
    node::NT
    out::OUT
end

function CachedNode(op, args...; kwargs...)
    node = ComputableNode(op, args, kwargs.data)
    out  = forward(node)
    CachedNode(node, out)
end

In [ ]:
arg(x::ComputableNode, i::Int) = x.args[i]
args(x::ComputableNode) = x.args
kwargs(x::ComputableNode) = x.kwargs
operator(x::ComputableNode) = x.f

arg(x::CachedNode, i::Int) = x.node.args[i]
args(x::CachedNode) = x.node.args
kwargs(x::CachedNode) = x.node.kwargs
operator(x::CachedNode) = x.node.f

In [ ]:
# definicja metody show, co ma się pokazać, gdy za parametr podamy jedną z nowo utworzonych struktur.
import Base: show
show(io::IO, x::Method)         = print(io, "fn ",  x.f);
show(io::IO, x::Operator)       = print(io, "op ",  x.f);
show(io::IO, x::Variable)       = print(io, "var ", x.value);
show(io::IO, x::CachedNode)     = print(io, "{cached (", x.node, ") => ", x.out, "}");
show(io::IO, x::ComputableNode) = print(io, "[", x.op, "](", x.args, ")");

In [ ]:
# przechodzenie grafu "w przód"
forward(x) = x
forward(leaf::LeafNode) = value(leaf)
forward(node::ComputableNode) = forward(node.op, map(forward, node.args)...; map(forward, node.kwargs)...)
forward(cached::CachedNode) = (cached.out = forward(cached.node))
forward(op::Broadcasted, args...) = Broadcast.broadcasted(op.f, args...)
forward(op::Operator, args...; kwargs...) = op.f(args...; kwargs...)
forward(x::NT) where {NT <: Node} = error("forward method is not implemented for node type: $NT")

In [ ]:
# funkcje potrzebne do korzystania z wartości stworzonych struktur
value(x) = x
value(x::Variable) = x.value
value(x::CachedNode) = value(x.out)
value(x::T) where {T <: Node} = error("Expected value in this node $x of type $T
 check if you defined a non-cached node
 or overload value function for your node.")

In [ ]:
# przechodzenie grafu "w tył"

function backward(x::Variable, grad)
    if isdefined(x, :grad)
        x.grad+= grad
    else
        x.grad = grad
    end
    nothing
end

function backward(node::CachedNode, f, grad)
    grad_inputs = gradient(node, grad)
    for (each, each_grad) in zip(args(node), grad_inputs)
        backward(each, each_grad)
    end
    nothing
end

function backward(cached::CachedNode, op::Broadcasted, grad)
    grad_inputs = gradient(cached, grad)
    for (each, each_grad) in zip(args(cached), grad_inputs)
        backward(each, each_grad)
    end
    nothing
end

backward(cached::CachedNode) = backward(cached, 1.0)
backward(cached::CachedNode, grad) = backward(cached, cached.node.op, grad)
backward(cached::CachedNode, op::Method, grad) = backward(cached, op.f, grad)
backward(cached::CachedNode, ::typeof(Broadcast.materialize), grad) = backward(arg(cached, 1), grad)

In [ ]:
# funkcje potrzebne do korzystania z gradientów struktur

gradient(x::CachedNode, grad) = gradient(x.node.op, grad, x.out, map(value, x.node.args)...; map(value, x.node.kwargs)...)
gradient(x::Operator,   grad, out, args...; kwargs...) = gradient(x.f, grad, out, args...; kwargs...)
gradient(op, grad, out, args...; kwargs...) = error("gradient of operator $op is not defined\n
 Possible Fix:\n
 define one of the following:\n
 1. gradient(::typeof($op), grad, out, args...; kwargs...)\n
 2. gradient(op::Method{typeof($op)}, grad, out, args...; kwargs...)\n
 3. gradient(op::Broadcasted{typeof($op)}, grad, out, args...; kwargs...)\n")

In [ ]:
# definicja podstawowych operatorów

import Base: +, -, *, /
+(x::Node) = CachedNode(+, x)
-(x::Node) = CachedNode(-, x)
gradient(::typeof(+), grad, output, x) = (grad * 1, )
gradient(::typeof(-), grad, output, x) = (grad *-1, )
+(x::Node, y::Node) = CachedNode(+, x, y)
-(x::Node, y::Node) = CachedNode(-, x, y)
*(x::Node, y::Node) = CachedNode(*, x, y)
/(x::Node, y::Node) = CachedNode(/, x, y)
gradient(::typeof(+), grad, output, x, y) = (grad * one(x),   grad * one(y))
gradient(::typeof(-), grad, output, x, y) = (grad * one(x),   grad *-one(y))
gradient(::typeof(*), grad, output, x, y) = (grad * y,        grad * x)
gradient(::typeof(/), grad, output, x, y) = (grad * one(x)/y, grad *-x/y/y)

In [ ]:
# definicja podstawowych funkcji i ich pochodnych
import Base: abs, sin, cos, tan, exp, sqrt, isless
abs(x::Node)  = CachedNode(abs, x)
sin(x::Node)  = CachedNode(sin, x)
cos(x::Node)  = CachedNode(cos, x)
tan(x::Node)  = CachedNode(tan, x)
exp(x::Node)  = CachedNode(exp, x)
sqrt(x::Node) = CachedNode(sqrt, x)
softmax(x::Node) = CachedNode(softmax, x)
isless(x::Node, y::Node) = isless(value(x), value(y))
gradient(::typeof(abs), grad, output, x)  = (grad * sign(x), )
gradient(::typeof(sin), grad, output, x)  = (grad * cos(x), )
gradient(::typeof(sink), grad, output, x)  = (grad *2*sin(x) * cos(x), )
gradient(::typeof(cos), grad, output, x)  = (grad *-sin(x), )
gradient(::typeof(tan), grad, output, x)  = (grad *(tan(x)^2 + 1), )
gradient(::typeof(exp), grad, output, x)  = (grad * exp(x), )
gradient(::typeof(sqrt), grad, output, x) = (grad * 0.5/sqrt(x), )
gradient(::typeof(tanh), grad, output, x)  = (grad * (1/cosh(x)^2), )
gradient(::typeof(cosh), grad, output, x)  = (grad * (sinh(x)), )
gradient(::typeof(softmax), grad, output, x)  = (grad * softmax_d(x), )
gradient(::typeof(RELU), grad, output, x)  = ifelse.(x<zero(x), (grad * zero(x), ), (grad * one(x), ))
gradient(::typeof(sinexp), grad, output, x) = (grad * exp(x) * cos(exp(x)), )
gradient(::typeof(relu), grad, output, x)  = (grad * relu_d(x), )
gradient(::typeof(σ), grad, output, x) = (grad * σ(x) *(1-σ(x)), )
gradient(::typeof(log), grad, output, x)  = (grad *(1/x), )

In [ ]:
# reguły konwersji i promocji
import Base: convert, promote_rule
convert(::Type{Variable{T}}, x::Number) where T   = Variable(convert(T, x))
convert(::Type{Variable{T}}, x::Float64) where T = Variable(convert(T, x), zero(T))
convert(::Type{Variable{T}}, x::Variable) where T = Variable(convert(T, x.value), convert(T, x.grad))

promote_rule(::Type{Variable{T}}, ::Type{R}) where {T,R} = Variable{promote_type(R,T)}

In [ ]:
struct ComputGraphStyle <: Broadcast.BroadcastStyle end
Base.BroadcastStyle(::Type{<:Node}) = ComputGraphStyle()
Broadcast.BroadcastStyle(s::ComputGraphStyle, x::Broadcast.BroadcastStyle) = s
Broadcast.broadcasted(::ComputGraphStyle, f, args...) = CachedNode(Broadcasted(f), args...)
Broadcast.broadcastable(x::Node) = x
Broadcast.materialize(x::Node) = CachedNode(Broadcast.materialize, x)
Base.similar(x::Node)                                      = Variable(similar(value(x)))
Base.similar(x::Node, dims::Dims)                          = Variable(similar(value(x), dims))
Base.similar(x::Node, eltype::Type{S}, dims::Dims) where S = Variable(similar(value(x), eltype, dims))

In [ ]:
# gradienty rejestrowanych operacji
gradient(::Broadcasted{typeof(+)}, grad, output, x)    = @. (grad * 1, )
gradient(::Broadcasted{typeof(-)}, grad, output, x)    = @. (grad *-1, )
gradient(::Broadcasted{typeof(+)}, grad, output, x, y) = @. (grad * one(x),   grad * one(y))
gradient(::Broadcasted{typeof(-)}, grad, output, x, y) = @. (grad * one(x),   grad *-one(y))
gradient(::Broadcasted{typeof(*)}, grad, output, x, y) = @. (grad * y,        grad * x)
gradient(::Broadcasted{typeof(/)}, grad, output, x, y) = @. (grad * one(x)/y, grad *-x/y/y)
gradient(::Broadcasted{typeof(abs)}, grad, output, x)  = @. (grad * sign(x), )
gradient(::Broadcasted{typeof(sin)}, grad, output, x)  = @. (grad *  cos(x),  )
gradient(::Broadcasted{typeof(sin)}, grad, output, x)  = @. (grad *  cos(x),  )
gradient(::Broadcasted{typeof(sink)}, grad, output, x)  = @. (grad *  2* sin(x) * cos(x),  )
gradient(::Broadcasted{typeof(cos)}, grad, output, x)  = @. (grad * -sin(x), )
gradient(::Broadcasted{typeof(tan)}, grad, output, x)  = @. (grad * (tan(x)^2 + 1), )
gradient(::Broadcasted{typeof(exp)}, grad, output, x)  = @. (grad *  exp(x), )
gradient(::Broadcasted{typeof(sqrt)}, grad, output, x) = @. (grad *.5/sqrt(x), )
gradient(::Broadcasted{typeof(tanh)}, grad, output, x)  = @. (grad * (1/cosh(x)^2), )
gradient(::Broadcasted{typeof(cosh)}, grad, output, x)  = @. (grad * (sinh(x)), )
gradient(::Broadcasted{typeof(softmax)}, grad, output, x)  = @. (grad * softmax_d(x), )
#gradient(::Broadcasted{typeof(RELU)}, grad, output, x)  = x.>zero(x) ? (@. (grad * zero(x), )) : (@. (grad * one(x), )) 
gradient(::Broadcasted{typeof(RELU)}, grad, output, x)  = ifelse.(x<zero(x), (@. (grad * zero(x), )),( @. (grad * one(x), ))) 
gradient(::Broadcasted{typeof(relu)}, grad, output, x)  = @. (grad * relu_d(x), )
gradient(::Broadcasted{typeof(sinexp)}, grad, output, x) = @. (grad * exp(x) * cos(exp(x)), )
gradient(::Broadcasted{typeof(σ)}, grad, output, x) = @. (grad * σ(x) *(1-σ(x)), )
gradient(::Broadcasted{typeof(log)}, grad, output, x)  = @. (grad * (1/x), )

# Weryfikacja różniczkowania w przód

In [ ]:
# funkcja, która pokazuje oryginalna funkcję i jej pochodną obliczoną za pomocą różniczkowania w przód
function forward_d(name, x)
    y = partials.(name.(x .+ϵ));
    yf = value.(name.(x.+ϵ));
    plot(x, yf, marker=:circle, markersize=2., label = "Original function")
    plot!(x, y, marker=:square, markersize=2., label = "Forward differentation")
    
    xaxis!("x")
    yaxis!("y")
end


##### W komórce poniżej należy wybrać jedną funkcję z dostępnych 

In [ ]:
x = collect(-10.0:0.1:+10.0)
forward_d(σ, x) # możliwe funkcje: sin, cos, tan, tanh, sink, sinexp, σ, itd...


# Weryfikacja różniczkowania w tył

##### W komórce poniżej należy wybrać jedną funkcję z dostępnych 

In [ ]:
# ukazanie funkcji i jej pochodnej wyliczonej za pomocą różniczkowania w tył
x = Variable(collect(-π:0.05:+π))
z = σ.(x) # możliwe funkcje: sin, cos, tan, tanh, sink, sinexp, σ, itd...
backward(z)
plot(value(x), value(z), label = "funkcja")
plot!(value(x), x.grad, label = "pochodna")


# Porównanie obu sposobów różniczkowania automatycznego

In [ ]:
f_do_porownania = σ # do wyboru sin, cos, sink, expk, sinexp, RELU, softmax

# zakres 
x = -π:0.01:+π
#x = 0.001:0.001:+1
# backward
w = Variable(collect(x))
q = f_do_porownania.(w)  # tutaj podmiana
#q = sin.(exp.(w))
backward(q) # uzupełnia wartości pochodnych

#forward

y = partials.(f_do_porownania.(x.+ϵ)); # TUTAJ PODMIANA
c = value.(f_do_porownania.(x.+ϵ))
zmienna = w.grad

plot(value(w), zmienna,  markersize=1., label = "Backward differentation")
plot!(x, y,  markersize=1., label = "Forward differentation")
plot!(x, c,  markersize=0.1, label = "Original function")
xaxis!("x")
yaxis!("y")

In [ ]:
# Sprawdzamy czy obie metody dały ten sam wynik 
roznica = zmienna - y;

In [ ]:
# Zgodnie z teorią jest linia na zerze - dokładnie takie same wyniki
plot(x, roznica, line=nothing, marker=:circle, markersize=1., ylims = (0, 1e-010), label = "Roznica wartości pochodnej i wartości funkcji")
xaxis!("x")
yaxis!("Błąd")

#### Porównanie wydajności

In [ ]:
# forward
@benchmark partials.(f_do_porownania.(x.+ϵ)) setup=(x=-5:0.01:+5)

In [ ]:
# backward
@benchmark backward(q) setup=(q = f_do_porownania.(w))

##### poniżej możliwość zapisania danych do pliku w celach dalszej obróbki

In [ ]:
# writedlm( "tanh.csv",  zmienna, ',') # należy odkomentować aby zapisać plik csv

#### Porównanie ilości kodu

In [ ]:
function code_forward(f, x)
    partials.(f.(x.+ϵ))
end

function code_backward(f, x)
    w = Variable(collect(x))
    q = f.(x)
    backward(q)
end


In [ ]:
x=-5:0.01:+5
@code_native code_backward(sin, x)

In [ ]:
# sprawdzenie wielokrotnego różniczkowania tej samej funkcji
D = derivative(f, x) = partials(f(Dual(x, one(x))))
f(x) = x^3
f′(x) = derivative(f,  x)
f″(x) = derivative(f′, x)
f‴(x) = derivative(f″, x)
x = -5:0.1:+5

plot(x,  f.(x),  label="x^3")
plot!(x, f′.(x), label="3x^2")
plot!(x, f″.(x), label="6x")
plot!(x, f‴.(x), label="6")

# Różniczkowanie funkcji wielu zmiennych 

#### Weryfikacja poprawności działania implementacji funkcji obliczających Jakobiany i Hesjany

In [ ]:
# definicja funkcji do liczenia macierzy Jacobiego
# za pomocą różniczkowania w przód 
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        column = partials.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

In [ ]:
# Funkcja obliczająca Jakobian z Jakobianu = Hesjan
H = function hessian(f, args::Vector)
    ∇f(x::Vector) = J(f, x)
    J(∇f, args)
end

In [ ]:
# Test działania funkcji wyliczającej Hesjan
f(x::Vector) = sum(cos, x) + prod(tanh, x) * sum(exp, x);
a = [0.6, 0.2, 0.3, 0.7, 0.68] ## lub a = rand(5)
H(f, a)
# -0.883588   0.712691   0.468028   0.179327   0.185705
#  0.712691  -1.01311    1.53009    0.595529   0.616022
#  0.468028   1.53009   -1.01615    0.390734   0.404258
#  0.179327   0.595529   0.390734  -0.813369   0.15411
#  0.185705   0.616022   0.404258   0.15411   -0.828181

In [ ]:
# Kolejny test działania funkcji wyliczającej Hesjan
f(x::Vector) = x[1]^5 + x[1]*x[2]^2
a = [5.0, 6.0] ## lub a = rand(2)
H(f, a)
# 2500.0  12.0
# 12.0  10.0

In [ ]:
# Test działania funkcji wyliczającej Jakobian
f(x::Vector) = [2x[1]*x[2]*4x[5], -x[4]/3x[2]*x[3]]
J(f, [1.0, 2.0, 3.0, 4.0, 5.0])
# 80.0  40.0   0.0        0.0  16.0
#  0.0   1.0  -0.666667  -0.5   0.0

# Porównanie dwóch trybów różniczkowania.

### Funkcje o wielu wejściach i wyjściach

In [ ]:
# biblioteki użyte do porównań
using ReverseDiff
using ForwardDiff

In [ ]:
# wektor wejściowy (maksymalnie będziemy liczyć pochodne funkcji o 101 zmiennych wejściowych)
u = Array{Float64,1}[]
for i in 1:101
   w = rand(i)
   push!(u, w)
end

In [ ]:
# definicja funkcji do różniczkowania
# nie było to pisane ręcznie ;)
# Arkusz excel za pomocą którego został wygenerowany kod został załączony razem z tym plikiem
func1(x...) = [2*x[1]-x[2]]
func2(x...) = [2*x[1]-x[2], 6x[2]*5x[1]]
func3(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-1*x[2]]
func4(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3]]
func5(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4]]
func6(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5]]
func7(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6]]
func8(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7]]
func9(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8]]
func10(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9]]
func11(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10]]
func12(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11]]
func13(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12]]
func14(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13]]
func15(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14]]
func16(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15]]
func17(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16]]
func18(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17]]
func19(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18]]
func20(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19]]
func21(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20]]
func22(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21]]
func23(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22]]
func24(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23]]
func25(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24]]
func26(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25]]
func27(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26]]
func28(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27]]
func29(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28]]
func30(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29]]
func31(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30]]
func32(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31]]
func33(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32]]
func34(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33]]
func35(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34]]
func36(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35]]
func37(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36]]
func38(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37]]
func39(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38]]
func40(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39]]
func41(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40]]
func42(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41]]
func43(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42]]
func44(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42], 1*x[44]-3*x[43]]
func45(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42], 1*x[44]-3*x[43], 3*x[45]-0*x[44]]
func46(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42], 1*x[44]-3*x[43], 3*x[45]-0*x[44], 7*x[46]-9*x[45]]
func47(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42], 1*x[44]-3*x[43], 3*x[45]-0*x[44], 7*x[46]-9*x[45], 6*x[47]-2*x[46]]
func48(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42], 1*x[44]-3*x[43], 3*x[45]-0*x[44], 7*x[46]-9*x[45], 6*x[47]-2*x[46], 9*x[48]-0*x[47]]
func49(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42], 1*x[44]-3*x[43], 3*x[45]-0*x[44], 7*x[46]-9*x[45], 6*x[47]-2*x[46], 9*x[48]-0*x[47], 0*x[49]-3*x[48]]
func50(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 6*x[3]-3*x[2], 6*x[4]-2*x[3], 6*x[5]-1*x[4], 8*x[6]-2*x[5], 7*x[7]-2*x[6], 0*x[8]-4*x[7], 8*x[9]-4*x[8], 5*x[10]-5*x[9], 0*x[11]-7*x[10], 4*x[12]-0*x[11], 7*x[13]-0*x[12], 5*x[14]-5*x[13], 8*x[15]-5*x[14], 6*x[16]-9*x[15], 7*x[17]-3*x[16], 1*x[18]-6*x[17], 7*x[19]-3*x[18], 8*x[20]-0*x[19], 2*x[21]-5*x[20], 2*x[22]-2*x[21], 9*x[23]-5*x[22], 9*x[24]-8*x[23], 1*x[25]-0*x[24], 4*x[26]-4*x[25], 0*x[27]-2*x[26], 3*x[28]-6*x[27], 6*x[29]-5*x[28], 8*x[30]-6*x[29], 6*x[31]-2*x[30], 6*x[32]-2*x[31], 8*x[33]-1*x[32], 4*x[34]-6*x[33], 1*x[35]-3*x[34], 4*x[36]-2*x[35], 9*x[37]-4*x[36], 2*x[38]-1*x[37], 9*x[39]-9*x[38], 2*x[40]-8*x[39], 3*x[41]-4*x[40], 2*x[42]-2*x[41], 7*x[43]-3*x[42], 1*x[44]-3*x[43], 3*x[45]-0*x[44], 7*x[46]-9*x[45], 6*x[47]-2*x[46], 9*x[48]-0*x[47], 0*x[49]-3*x[48], 2*x[50]-7*x[49]]
func51(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50]]
func52(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51]]
func53(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52]]
func54(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53]]
func55(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54]]
func56(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55]]
func57(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56]]
func58(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57]]
func59(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58]]
func60(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59]]
func61(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60]]
func62(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61]]
func63(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62]]
func64(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63]]
func65(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64]]
func66(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65]]
func67(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66]]
func68(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67]]
func69(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68]]
func70(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69]]
func71(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70]]
func72(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71]]
func73(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72]]
func74(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73]]
func75(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74]]
func76(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75]]
func77(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76]]
func78(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77]]
func79(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78]]
func80(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79]]
func81(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80]]
func82(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81]]
func83(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82]]
func84(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83]]
func85(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84]]
func86(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85]]
func87(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86]]
func88(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87]]
func89(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88]]
func90(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89]]
func91(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90]]
func92(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91]]
func93(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92]]
func94(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93]]
func95(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93], 2*x[95]-2*x[94]]
func96(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93], 2*x[95]-2*x[94], 1*x[96]-7*x[95]]
func97(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93], 2*x[95]-2*x[94], 1*x[96]-7*x[95], 8*x[97]-4*x[96]]
func98(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93], 2*x[95]-2*x[94], 1*x[96]-7*x[95], 8*x[97]-4*x[96], 9*x[98]-6*x[97]]
func99(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93], 2*x[95]-2*x[94], 1*x[96]-7*x[95], 8*x[97]-4*x[96], 9*x[98]-6*x[97], 0*x[99]-6*x[98]]
func100(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93], 2*x[95]-2*x[94], 1*x[96]-7*x[95], 8*x[97]-4*x[96], 9*x[98]-6*x[97], 0*x[99]-6*x[98], 2*x[100]-5*x[99]]
func101(x...) = [2*x[1]-x[2], 6x[2]*5x[1], 3*x[3]-1*x[2], 0*x[4]-6*x[3], 4*x[5]-3*x[4], 1*x[6]-3*x[5], 9*x[7]-4*x[6], 8*x[8]-9*x[7], 0*x[9]-7*x[8], 1*x[10]-5*x[9], 8*x[11]-1*x[10], 7*x[12]-5*x[11], 2*x[13]-2*x[12], 3*x[14]-9*x[13], 8*x[15]-5*x[14], 8*x[16]-0*x[15], 3*x[17]-5*x[16], 0*x[18]-4*x[17], 8*x[19]-3*x[18], 2*x[20]-9*x[19], 3*x[21]-5*x[20], 4*x[22]-6*x[21], 6*x[23]-6*x[22], 8*x[24]-6*x[23], 7*x[25]-0*x[24], 9*x[26]-5*x[25], 9*x[27]-1*x[26], 3*x[28]-0*x[27], 4*x[29]-4*x[28], 7*x[30]-6*x[29], 5*x[31]-3*x[30], 9*x[32]-5*x[31], 0*x[33]-7*x[32], 2*x[34]-3*x[33], 4*x[35]-2*x[34], 2*x[36]-9*x[35], 3*x[37]-2*x[36], 7*x[38]-8*x[37], 3*x[39]-6*x[38], 8*x[40]-6*x[39], 0*x[41]-8*x[40], 1*x[42]-5*x[41], 7*x[43]-8*x[42], 2*x[44]-7*x[43], 7*x[45]-1*x[44], 1*x[46]-7*x[45], 4*x[47]-1*x[46], 2*x[48]-1*x[47], 1*x[49]-8*x[48], 0*x[50]-1*x[49], 0*x[51]-2*x[50], 0*x[52]-0*x[51], 5*x[53]-8*x[52], 0*x[54]-5*x[53], 2*x[55]-9*x[54], 6*x[56]-3*x[55], 1*x[57]-2*x[56], 8*x[58]-5*x[57], 8*x[59]-9*x[58], 2*x[60]-1*x[59], 0*x[61]-5*x[60], 7*x[62]-6*x[61], 1*x[63]-5*x[62], 0*x[64]-3*x[63], 9*x[65]-0*x[64], 1*x[66]-4*x[65], 5*x[67]-9*x[66], 7*x[68]-6*x[67], 6*x[69]-8*x[68], 3*x[70]-0*x[69], 1*x[71]-9*x[70], 2*x[72]-2*x[71], 6*x[73]-7*x[72], 9*x[74]-5*x[73], 3*x[75]-4*x[74], 4*x[76]-3*x[75], 3*x[77]-1*x[76], 3*x[78]-2*x[77], 8*x[79]-6*x[78], 2*x[80]-4*x[79], 7*x[81]-1*x[80], 2*x[82]-3*x[81], 7*x[83]-8*x[82], 5*x[84]-1*x[83], 6*x[85]-7*x[84], 1*x[86]-6*x[85], 3*x[87]-9*x[86], 0*x[88]-6*x[87], 4*x[89]-4*x[88], 4*x[90]-3*x[89], 8*x[91]-9*x[90], 2*x[92]-3*x[91], 7*x[93]-0*x[92], 3*x[94]-4*x[93], 2*x[95]-2*x[94], 1*x[96]-7*x[95], 8*x[97]-4*x[96], 9*x[98]-6*x[97], 0*x[99]-6*x[98], 2*x[100]-5*x[99], 1*x[101]-5*x[100]]


In [ ]:
# tablica z funkcjami
# również generowana z pomocą Excela
fn = [func2, func3, func4, func5, func6, func7, func8, func9, func10, func11, func12, func13, func14, func15, func16, func17, func18, func19, func20, func21, func22, func23, func24, func25, func26, func27, func28, func29, func30, func31, func32, func33, func34, func35, func36, func37, func38, func39, func40, func41, func42, func43, func44, func45, func46, func47, func48, func49, func50, func51, func52, func53, func54, func55, func56, func57, func58, func59, func60, func61, func62, func63, func64, func65, func66, func67, func68, func69, func70, func71, func72, func73, func74, func75, func76, func77, func78, func79, func80, func81, func82, func83, func84, func85, func86, func87, func88, func89, func90, func91, func92, func93, func94, func95, func96, func97, func98, func99, func100, func101]

In [ ]:
# puste tablice na wyniki
times_forward = []
allocs_forward = []
times_reverse = []
allocs_reverse = []

In [ ]:
# funkcja, za pomocą której przebiegać będzie każde różniczkowanie w przód
function goForward(f, iterator)
    m = @benchmark ForwardDiff.jacobian(x-> f(x...), u[iterator])
    append!(times_forward, mean(m.times))
    append!(allocs_forward, m.allocs)
end

In [ ]:
# funkcja, za pomocą której przebiegać będzie każde różniczkowanie w tył
function goReverse(f, iterator)
    m = @benchmark ReverseDiff.jacobian(x-> f(x...), u[iterator])
    append!(times_reverse, mean(m.times))
    append!(allocs_reverse, m.allocs)
end

In [ ]:
#wykonanie obliczeń każdego ze 100 Jakobianów
iterator = 2
for fff in fn
    goForward(fff, iterator)
    goReverse(fff, iterator)
    iterator = iterator + 1
end

In [ ]:
# export do csv w przypadku potrzeby dalszej obróbki danych
# writedlm( "reverse_times.csv",  times_reverse, ',')
# writedlm( "forward_times.csv",  times_forward, ',')
# writedlm( "reverse_allocs.csv",  allocs_reverse, ',')
# writedlm( "forward_allocs.csv",  allocs_forward, ',')

In [ ]:
# tworzenie wykresu średniego czasu wykonywania obliczeń
plot(d, times_reverse/1000, label = "Różniczkowanie trybem w tył")
plot!(d, times_forward/1000, label = "Różniczkowanie trybem w przód")
xlabel!("Rozmiar macierzy Jacobiego")
ylabel!("Czas wykonywania obliczeń [μs]")
#savefig("myplot.png")

In [ ]:
# tworzenie wykresu liczby alokacji
plot(d, allocs_reverse, label = "Liczba alokacji, tryb w tył")
plot!(d, allocs_forward, label = "Liczba alokacji, tryb w przód")
xlabel!("Rozmiar macierzy Jacobiego")
ylabel!("Liczba alokacji")
#savefig("allocs.png")

# Implementacja prostej sieci neuronowej

In [ ]:
# funkcja do tworzenia macierzy jednostkowej
eye(n::Integer) = Matrix(1.0I, n, n);

In [ ]:
# funkcja obliczająca griadienty E względem wag (metoda ręczna, należy znać strukturę sieci)
function ∇W(x, x̂, ŷ, y, Wo)
    Eŷ  = ŷ - y
    ŷȳ  = ŷ  |> length |> eye
    ȳWo = x̂  |> transpose
    x̄Wh = x  |> transpose
    ȳx̂  = Wo |> transpose
    x̂x̄  = x̂  .* (1.0 .- x̂) |> diagonal
    Eȳ  = ŷȳ * Eŷ
    Ex̂  = ȳx̂ * Eȳ
    Ex̄  = x̂x̄ * Ex̂
    EWo = Eȳ * ȳWo
    EWh = Ex̄ * x̄Wh
    return EWo, EWh
end

In [ ]:
mean_squared_loss(y::Vector, ŷ::Vector) = sum(0.5(y - ŷ).^2) # definicja funkcji błędu
fullyconnected(w::Vector, n::Number, m::Number, v::Vector, activation::Function) = activation.(reshape(w, n, m) * v)
# Inicjalizacja wag
Wh  = randn(10,2)
Wo  = randn(1,10)
dWh = similar(Wh)
dWo = similar(Wo)
x = [2.58;4.22]
y = [0.125]
E = Float64[]

# stworzenie sieci
function net(x, wh, wo, y)
    x̂ = fullyconnected(wh, 10, 2, x, σ)
    ŷ = fullyconnected(wo, 1, 10, x̂, u->u)
        
    E = mean_squared_loss(y, ŷ)
end
Ei = net(x, Wh[:], Wo[:], y)
println("Initial loss: ", Ei)

In [ ]:
dnet_Wh(x, wh, wo, y) = J(w -> net(x, w, wo, y), wh);
dnet_Wo(x, wh, wo, y) = J(w -> net(x, wh, w, y), wo);

In [ ]:
# przebieg uczenia
iterations = 100  # należy określić ilość iteracji

for i in 1:iterations
    push!(E, Ei)
    dWh[:] = dnet_Wh(x, Wh[:], Wo[:], y);
    dWo[:] = dnet_Wo(x, Wh[:], Wo[:], y);
    Wh -= 0.1dWh
    Wo -= 0.1dWo
    Ei  = net(x, Wh[:], Wo[:], y)
end

In [ ]:
# wyrysowanie wykresu
plot(log10.(E), label="Loss function", markersize = 2, marker=:square)
xaxis!("epoch")
yaxis!("log(loss function)")

Źródła

- https://github.com/JuliaAcademy/JuliaAcademyMaterials/blob/main/Courses/Foundations%20of%20machine%20learning/20.Automatic-Differentiation-in-10-Minutes.jl
- https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
- https://github.com/bchaber/1di2153/blob/main/W03/reverse.ipynb
- https://github.com/Roger-luo/YAAD.jl
- https://www.youtube.com/watch?v=vAp6nUMrKYg
- https://github.com/bchaber
- https://www.geeksforgeeks.org/
- https://discourse.julialang.org/
- https://medium.com/@bowlescompling/m2-1-softmax-in-julia-1498901f741c
- https://int8.io/neural-networks-in-julia-hyperbolic-tangent-and-relu/
- https://arxiv.org/abs/1811.05031v2
- https://github.com/JuliaDiff/ReverseDiff.jl
- https://github.com/JuliaDiff/ForwardDiff.jl